# Semantic Search on PDF Documents with qFlat Index

##### Note: This example requires a KDB.AI endpoint and API key. Sign up for a free [KDB.AI account](https://kdb.ai/get-started).

This example demonstrates how to use KDB.AI to run semantic search on unstructured text documents.

<div class="alert alert-block alert-info">
<b>Tip:</b> This sample uses ‘qFlat’ , a new vector index choice in KDB.AI. It will support the same API options as the existing ‘Flat’ index but with the significant difference that the index is stored on-disk and memory-mapped as required. This means data inserts will have negligible memory and cpu footprints. The vector index can grow and be searched as long as there is disk space available and works great for datasets with up to 1,000,000 vectors. Among other cases, this stands out as a great index for memory contrained situations such as edge devices.
</div>

Semantic search allows users to perform searches based on the meaning or similarity of the data rather than exact matches. It works by converting the query into a vector representation and then finding similar vectors in the database. This way, even if the query and the data in the database are not identical, the system can identify and retrieve the most relevant results based on their semantic meaning.

### Aim
In this tutorial, we'll walk you through the process of performing semantic search on documents, taking PDFs as example, using KDB.AI as the vector store. We will cover the following topics:

0. Load PDF Data
1. Create Sentence Vector Embeddings
2. Store Embeddings in KDB.AI
3. Search For Similar Sentences To A Target Sentence
4. Delete the KDB.AI Table

---

## 0. Setup

### Install dependencies

In order to successfully run this sample, note the following steps depending on where you are running this notebook:

-***Run Locally / Private Environment:*** The [Setup](https://github.com/KxSystems/kdbai-samples/blob/main/README.md#setup) steps in the repository's `README.md` will guide you on prerequisites and how to run this with Jupyter.


-***Colab / Hosted Environment:*** Open this notebook in Colab and run through the cells.

### Set Environment Variables

In [ ]:
%pip install kdbai_client pypdf sentence_transformers

In [ ]:
### !!! Only run this cell if you need to download the data into your environment, for example in Colab
### This downloads research paper pdf into your environment
!mkdir ./data
!wget -P ./data https://raw.githubusercontent.com/KxSystems/kdbai-samples/main/document_search/data/research_paper.pdf

In [3]:
import os

In [4]:
### ignore tensorflow warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

### Import Packages

In [5]:
# load data
import pypdf
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# embeddings
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

In [7]:
# vector DB
import os
import kdbai_client as kdbai
from getpass import getpass
import time

### Configure Console

In [8]:
pd.set_option("display.max_colwidth", 300)

### Define Helper Functions

In [9]:
def show_df(df: pd.DataFrame) -> pd.DataFrame:
    print(df.shape)
    return df.head()

## 1. Load PDF Data

### Read Text From PDF Document

We leverage the power of PyPDF2 for PDF processing and `nltk` for advanced natural language processing. The code below extracts content from each page of the PDF and processes it to identify sentences.

The PDF we are using is [this research paper](https://arxiv.org/pdf/2308.05801.pdf) presenting information on the formation of Interstellar Objects in the Milky Way.

In [10]:
# Read PDF file
with open("data/research_paper.pdf", "rb") as pdf_file:
    pdf_pages = pypdf.PdfReader(pdf_file).pages
    page_list = [page.extract_text() for page in pdf_pages]

In [11]:
# Concatenate text from each page
full_pdf_text = "".join(page_list)

### Split The Text Into Sentences

<div class="alert alert-block alert-warning">
    <b>Note: </b>
    Before running the following line of code, please ensure that you have installed the English sentence tokenizer as stated in the `README.md` file in this repository.
</div>

In [12]:
# Split the PDF into sentences
pdf_sentences = sent_tokenize(full_pdf_text)
len(pdf_sentences)

591

In [13]:
pdf_sentences[0]

'Draft version August 14, 2023\nTypeset using L ATEX default style in AASTeX631\nThe Galactic Interstellar Object Population: A Framework for Prediction and Inference\nMatthew J. Hopkins\n ,1Chris Lintott\n ,1Michele T. Bannister\n ,2J.'

## 2. Create Sentence Vector Embeddings

Next, we use the Sentence Transformers library to create embeddings for our collection of sentences.

### Selecting a Sentence Transformer Model

There are 100+ of different types of Sentence Transformers models available - see [HuggingFace](https://huggingface.co/sentence-transformers) for the full list. The diversity among these primarily stems from variations in their training data. Selecting the ideal model for your needs involves matching the domain and task closely, while also considering the benefits of incorporating larger datasets to enhance scale.

This tutorial will use the `all-MiniLM-L6-v2` pre-trained model. This embedding model can create sentence and document embeddings that can be used for a wide variety of tasks including semantic search which makes it a good choice for our needs.

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

### Generate Sentence Embeddings Using This Model

We prepare embeddings by applying the sentence transformer model to our sentences to encode them. The we do some transformation to get this into DataFrame which is the format accepted by KDB.AI.

In [15]:
# Create embeddings
embeddings_list = model.encode(np.array(pdf_sentences)).tolist()
embeddings_df = pd.DataFrame({"vectors": embeddings_list, "sentences": pdf_sentences})

In [16]:
show_df(embeddings_df)

(591, 2)


,vectors,sentences
0,"[-0.059804268181324005, -0.09221810102462769, 0.058069996535778046, 0.06884294003248215, -0.0030452304054051638, 0.007304240483790636, -0.028959110379219055, 0.0595787838101387, 0.017943743616342545, 0.043501030653715134, 0.005012953653931618, -0.07875007390975952, -0.02570340223610401, -0.04147...","Draft version August 14, 2023\nTypeset using L ATEX default style in AASTeX631\nThe Galactic Interstellar Object Population: A Framework for Prediction and Inference\nMatthew J. Hopkins\n ,1Chris Lintott\n ,1Michele T. Bannister\n ,2J."
1,"[-0.08154530823230743, -0.11342314630746841, 0.08425560593605042, 0.08849422633647919, 0.024723634123802185, -0.08773960173130035, -0.06128991022706032, 0.0106121264398098, 0.06387822329998016, 0.021902449429035187, -0.033225465565919876, -0.09321605414152145, -0.04425090178847313, -0.0568475760...","Ted Mackereth\n ,3, 4, 5, ∗and\nJohn C. Forbes\n2\n1Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK\n2School of Physical and Chemical Sciences—Te Kura Mat¯ u, University of Canterbury, Private Bag 4800, Christchurch 8140, New Zealand\n3Just ..."
2,"[-0.07799109816551208, -0.08398666232824326, 0.02723710611462593, 0.09858055412769318, 0.010515090078115463, -0.01670389622449875, -0.055797528475522995, 0.027984360232949257, 0.03926650434732437, 0.03248238563537598, 0.03837994113564491, -0.12464361637830734, -0.012645130045711994, -0.018288377...","We define a novel framework: firstly to predict\nthe properties of this Galactic ISO population by combining models of processes across planetary\nand galactic scales, and secondly to make inferences about the processes modelled, by comparing the\npredicted population to what is observed."
3,"[-0.05338575318455696, -0.037147365510463715, 0.07784582674503326, 0.05165949463844299, -0.07674478739500046, -0.007766904775053263, 0.008464669808745384, 0.04289115220308304, 0.04356798157095909, 0.06012833118438721, 0.04407043382525444, -0.10177762806415558, 0.030359704047441483, -0.0340371690...",We predict the spatial and compositional distribution of the\nGalaxy’s population of ISOs by modelling the Galactic stellar population with data from the APOGEE\nsurvey and combining this with a protoplanetary disk chemistry model.
4,"[-0.06326982378959656, -0.01720457151532173, 0.047737207263708115, 0.08325360715389252, 0.03296627104282379, -0.0660020187497139, 0.0015915816184133291, 0.003246008651331067, 0.03456269949674606, -0.020524047315120697, 0.041225481778383255, -0.15274153649806976, -0.011771060526371002, 0.07314326...","Selecting ISO water mass\nfraction as an example observable quantity, we evaluate its distribution both at the position of the Sun\nand averaged over the Galactic disk; our prediction for the Solar neighbourhood is compatible with the\ninferred water mass fraction of 2I/Borisov."


It is important to note the dimension of our embeddings is 384. This will need to match the dimensions we set in the KDB.AI index in the next step. We can easily check this using `len` to count elements in our vector.

In [17]:
len(embeddings_df["vectors"][0])

384

## 3. Store Embeddings in KDB.AI

With the embeddings created, we need to store them in a vector database to enable efficient searching.

### Define KDB.AI Session

KDB.AI comes in two offerings:

1. [KDB.AI Cloud](https://trykdb.kx.com/kdbai/signup/) - For experimenting with smaller generative AI projects with a vector database in our cloud.
2. [KDB.AI Server](https://trykdb.kx.com/kdbaiserver/signup/) - For evaluating large scale generative AI applications on-premises or on your own cloud provider.

Depending on which you use there will be different setup steps and connection details required.

##### Option 1. KDB.AI Cloud

To use KDB.AI Cloud, you will need two session details - a URL endpoint and an API key.
To get these you can sign up for free [here](https://trykdb.kx.com/kdbai/signup).

You can connect to a KDB.AI Cloud session using `kdbai.Session` and passing the session URL endpoint and API key details from your KDB.AI Cloud portal.

If the environment variables `KDBAI_ENDPOINTS` and `KDBAI_API_KEY` exist on your system containing your KDB.AI Cloud portal details, these variables will automatically be used to connect.
If these do not exist, it will prompt you to enter your KDB.AI Cloud portal session URL endpoint and API key details.

In [ ]:
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else getpass("KDB.AI API key: ")
)

In [19]:
session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

##### Option 2. KDB.AI Server

To use KDB.AI Server, you will need download and run your own container.
To do this, you will first need to sign up for free [here](https://trykdb.kx.com/kdbaiserver/signup/).

You will receive an email with the required license file and bearer token needed to download your instance.
Follow instructions in the signup email to get your session up and running.

Once the [setup steps](https://code.kx.com/kdbai/gettingStarted/kdb-ai-server-setup.html) are complete you can then connect to your KDB.AI Server session using `kdbai.Session` and passing your local endpoint.

In [ ]:
# session = kdbai.Session(endpoint="http://localhost:8082")

### Define Vector DB Table Schema

The next step is to define a schema for our KDB.AI table where we will store our embeddings. Our table will have two columns.

At this point you will select the index and metric you want to use for searching.

With KDB.AI we have the choice between HNSW (Hierarchical Navigable Small World), IVF, IVFPQ and Flat indexing methods. Generally, for semantic search of documents, the HNSW indexing method might be more suitable. Here's why:

- **Search Speed and Approximation**: HNSW is designed for fast approximate nearest neighbor searches. It can efficiently handle high-dimensional data, which is common in natural language processing tasks involving text documents.
- **Semantic Representation**: The Sentence Transformers library, used in this example, generates embeddings that capture semantic meaning. HNSW is well-suited for indexing such embeddings and performing semantic searches.
- **Scalability**: HNSW is scalable and can handle large datasets effectively, making it suitable for applications with a vast number of documents.

HNSW provides approximate search results, meaning that the nearest neighbors might not be exact matches but are close in terms of similarity.

In [20]:
pdf_schema = {
    "columns": [
        {"name": "sentences", "pytype": "str"},
        {
            "name": "vectors",
            "pytype": "float64",
            "vectorIndex": {"dims": 384, "metric": "L2", "type": "qFlat"},
        },
    ]
}

### Create Vector DB Table

Use the KDB.AI `create_table` function to create a table that matches the defined schema in the vector database.

In [21]:
# First ensure the table does not already exist
try:
    session.table("pdf").drop()
    time.sleep(5)
except kdbai.KDBAIException:
    pass

In [22]:
table = session.create_table("pdf", pdf_schema)

We can use `query` to see our table exists but is empty.

In [23]:
table.query()

,sentences,vectors


### Add Embedded Data to KDB.AI Table

In [24]:
table.insert(embeddings_df)

'Insert successful'

### Verify Data Has Been Inserted

Running `table.query()` should show us that data has been added.

In [25]:
show_df(table.query())

(591, 2)


,vectors,sentences
0,"[-0.05980427, -0.0922181, 0.058069997, 0.06884294, -0.0030452304, 0.0073042405, -0.02895911, 0.059578784, 0.017943744, 0.04350103, 0.0050129537, -0.078750074, -0.025703402, -0.04147788, -0.03986909, -0.006262344, -0.002314164, -0.08159439, 0.028753396, 0.039623264, 0.06351384, 0.04714837, 0.0192...","Draft version August 14, 2023\nTypeset using L ATEX default style in AASTeX631\nThe Galactic Interstellar Object Population: A Framework for Prediction and Inference\nMatthew J. Hopkins\n ,1Chris Lintott\n ,1Michele T. Bannister\n ,2J."
1,"[-0.08154531, -0.11342315, 0.084255606, 0.08849423, 0.024723634, -0.0877396, -0.06128991, 0.010612126, 0.06387822, 0.02190245, -0.033225466, -0.093216054, -0.0442509, -0.056847576, 0.049649086, -0.057687998, 0.037710313, -0.10215939, 0.00079446856, 0.056125678, -0.034007806, 0.044451304, -0.0385...","Ted Mackereth\n ,3, 4, 5, ∗and\nJohn C. Forbes\n2\n1Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK\n2School of Physical and Chemical Sciences—Te Kura Mat¯ u, University of Canterbury, Private Bag 4800, Christchurch 8140, New Zealand\n3Just ..."
2,"[-0.0779911, -0.08398666, 0.027237106, 0.098580554, 0.01051509, -0.016703896, -0.05579753, 0.02798436, 0.039266504, 0.032482386, 0.03837994, -0.12464362, -0.01264513, -0.018288378, 0.0044818097, -0.12488365, -0.03791929, -0.052581396, -0.008583976, -0.01311017, -0.031446036, 0.06907686, -0.05145...","We define a novel framework: firstly to predict\nthe properties of this Galactic ISO population by combining models of processes across planetary\nand galactic scales, and secondly to make inferences about the processes modelled, by comparing the\npredicted population to what is observed."
3,"[-0.053385753, -0.037147366, 0.07784583, 0.051659495, -0.07674479, -0.007766905, 0.00846467, 0.042891152, 0.04356798, 0.06012833, 0.044070434, -0.10177763, 0.030359704, -0.03403717, 0.043043666, -0.09360116, -0.02031162, -0.04007734, 0.06953779, 0.031602092, 0.0008737179, 0.06921691, -0.05480820...",We predict the spatial and compositional distribution of the\nGalaxy’s population of ISOs by modelling the Galactic stellar population with data from the APOGEE\nsurvey and combining this with a protoplanetary disk chemistry model.
4,"[-0.063269824, -0.017204572, 0.047737207, 0.08325361, 0.03296627, -0.06600202, 0.0015915816, 0.0032460087, 0.0345627, -0.020524047, 0.04122548, -0.15274154, -0.0117710605, 0.073143266, -0.011138947, -0.10220338, -0.010274912, 0.0364803, 0.005580916, 0.026839772, -0.0134625975, 0.050991774, 0.015...","Selecting ISO water mass\nfraction as an example observable quantity, we evaluate its distribution both at the position of the Sun\nand averaged over the Galactic disk; our prediction for the Solar neighbourhood is compatible with the\ninferred water mass fraction of 2I/Borisov."


## 4. Search For Similar Sentences To A Target Sentence

Now that the embeddings are stored in KDB.AI, we can perform semantic search using `search`.

### Search 1

First, we embed our search term using the Sentence Transformer model as before. Then we search our index to return the three most similar vectors.

In [26]:
search_term1 = "number of interstellar objects in the milky way"

In [27]:
encoded_search_term1 = model.encode(search_term1).tolist()

In [28]:
results1 = table.search([encoded_search_term1], n=3)
results1[0]

,__nn_distance,vectors,sentences
0,0.642638,"[-0.08154531, -0.11342315, 0.084255606, 0.08849423, 0.024723634, -0.0877396, -0.06128991, 0.010612126, 0.06387822, 0.02190245, -0.033225466, -0.093216054, -0.0442509, -0.056847576, 0.049649086, -0.057687998, 0.037710313, -0.10215939, 0.00079446856, 0.056125678, -0.034007806, 0.044451304, -0.0385...","Ted Mackereth\n ,3, 4, 5, ∗and\nJohn C. Forbes\n2\n1Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK\n2School of Physical and Chemical Sciences—Te Kura Mat¯ u, University of Canterbury, Private Bag 4800, Christchurch 8140, New Zealand\n3Just ..."
1,0.668817,"[-0.08688256, -0.023635495, 0.06744848, 0.08018778, -0.059747145, -0.03222732, -0.00877004, 0.04886193, -0.051810436, 0.017594708, 0.02430901, -0.091474384, 0.03819925, -0.061818995, -0.0016515158, 0.0011128797, 0.01666607, -0.08598414, 0.06537522, 0.034577794, 0.011868171, 0.045408193, -0.00955...","In this work, we develop\nthis method and apply it to the stellar population of the Milky Way, estimated with data from the APOGEE survey, to\npredict a broader set of properties of our own Galaxy’s population of interstellar objects."
2,0.741215,"[-0.07292614, 0.021450141, 0.016602317, 0.05125197, -0.017564908, -0.0874388, -0.020641718, 0.037198327, -0.024873124, 0.039150987, 0.043217096, -0.13203551, -0.033658065, -0.06451232, -0.004239228, 0.00060118927, 0.0344788, -0.049100593, 0.0023647982, 0.061882924, 0.037032355, 0.11182225, -0.05...","Keywords: Interstellar objects (52), Small Solar System bodies(1469), Galaxy Evolution (594)\n1.INTRODUCTION\n1I/‘Oumuamua (Meech et al."


The results returned from `table.search` show the closest matches along with value of nearest neighbor distances `nn_distance`.
We can see these sentences do reference our search term 'number of interstellar objects in the milky way' in some way.

### Search 2

Let's try another search term.

In [29]:
search_term2 = "how does planet formation occur"

In [30]:
encoded_search_term2 = model.encode(search_term2).tolist()

In [31]:
results2 = table.search([encoded_search_term2], n=3)
results2[0]

,__nn_distance,vectors,sentences
0,0.907236,"[-0.07665148, -0.065820254, 0.03430574, 0.026705816, 0.07752172, -0.05098922, 0.00799623, 0.02346398, 0.09635236, 0.058903504, -0.009348639, -0.049475733, 0.040722128, -0.08664801, 0.05520128, -0.004725993, -0.06510683, -0.051167995, -0.007605029, 0.022087516, 0.06251535, -0.07805417, -0.0462687...","The pop-\nulation’s dominant dynamical formation mechanisms would preferentially harvest more distant, ice-rich planetesimals\nfrom the disks of the source systems."
1,0.932817,"[-0.011731947, -0.06267538, 0.08392808, -0.0363365, -0.002124414, -0.050823975, -0.00048589325, -0.027594214, 0.13681786, 0.06662702, -0.026512465, -0.01938669, 0.016084943, -0.08345804, 0.06584347, -0.10745433, -0.031991202, -0.09809714, 0.03796809, 0.05513507, 0.041274864, 0.045654308, -0.0798...","A protoplanetary disk has to first order the same composition as the star it forms around,\nsince they both form from the same molecular cloud core."
2,0.954052,"[-0.08789742, -0.05399665, 0.08170222, 0.050576083, -0.0012671131, -0.051809072, -0.036422946, 0.014396563, 0.10822276, 0.043686096, -0.0799644, -0.070625275, 0.053738702, -0.06110463, 0.033653535, -0.08810672, -0.122173086, -0.025936356, -0.0060172374, -0.023666425, 0.05693163, -0.03541065, -0....","While in reality, stars will each produce a distribution of ISOs that\nformed at different positions in their protoplanetary disk and thus have a range of compositions, this simplification\nof only modelling planetesimals which form exterior to the water ice line is justified by the proportional..."


Again, we can see these sentences do reference our search term 'how does planet formation occur' in some way.

## 5. Delete the KDB.AI Table

Once finished with the table, it is best practice to drop it.

In [32]:
table.drop()

True

## Take Our Survey

We hope you found this sample helpful! Your feedback is important to us, and we would appreciate it if you could take a moment to fill out our brief survey. Your input helps us improve our content.

[**Take the Survey**](https://delighted.com/t/ejgOzTpo)